In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter

import numpy as np
from tqdm import tqdm

import os
import shutil
import math

from utils.post_process import cal_absolute_from_relative, plot_from_pose
from utils.misc import to_var, adjust_learning_rate, pre_create_file_train, pre_create_file_test, \
    display_loss_tb, display_loss_tb_val

##### load path

In [2]:
dir_data = '/media/czy/DATA/Share/Kitti/color'
dir_label = './dataset'
model_dir = './model'
log_dir = './log'
args_phase = 'Train'
args_beta = 10
args_epoch_save  = 2

In [3]:
from net.cnn import Net
from dataset.kitti import KITTIDataSet

In [4]:
torch.set_default_tensor_type('torch.FloatTensor')
model = Net()
if torch.cuda.is_available():
    print("Using GPU Device!")
    model = nn.DataParallel(model.cuda())
else:
    print("Using CPU Device!")

/home/czy/Projects/pyproject/vo-demo/net/cnn.py:68: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  xavier_normal(m.weight.data)


Using GPU Device!


In [5]:
dir_models = model_dir + '/cnn'
dir_logs = log_dir + '/cnn'
dir_model = dir_models + '/' + '20220321'
dir_log = dir_logs + '/' + '20220321'
# if not os.path.exists(dir_models):
#     os.mkdir(dir_models)
# if not os.path.exists(dir_logs):
#     os.mkdir(dir_logs)
# if not os.path.exists(dir_model):
#     os.mkdir(dir_model)
# if os.path.exists(dir_log):
#     shutil.rmtree(dir_log)
# os.mkdir(dir_log)


In [6]:
writer = SummaryWriter(dir_log)
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)

In [23]:
data_set_t = KITTIDataSet(dir_data=dir_data,dir_label=dir_label,samples='i0',phase='Train')
data_set_v = KITTIDataSet(dir_data=dir_data, dir_label=dir_label, phase='Valid')
loader_t = DataLoader(data_set_t, batch_size=2, shuffle=True, num_workers=1)
loader_v = DataLoader(data_set_v, batch_size=2, shuffle=False, num_workers=1)

In [24]:
step_per_epoch = int(math.floor(len(data_set_t) / loader_t.batch_size))
step_val = int(math.floor(step_per_epoch / 3)) 
step_per_epoch,step_val


(2270, 756)

In [25]:
def adjust_learning_rate(optimizer, epoch, lr_base, gamma=0.316, epoch_lr_decay=25):
    """
    自动调整学习率
    :param optimizer: 优化方法
    :param epoch: 迭代次数
    :param lr_base: 基础学习率
    :param gamma: gamma值
    :param epoch_lr_decay: 速率衰减次数
    :return:
    """
    """
        epoch       lr
        000-025     1e-4
        025-050     3e-5
        050-075     1e-5
        075-100     3e-6
        100-125     1e-6
        125-150     3e-7
    """

    exp = int(math.floor(epoch / epoch_lr_decay))
    lr_decay = gamma ** exp
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr_decay * lr_base

In [26]:
for epoch in np.arange(5):
    adjust_learning_rate(optimizer, epoch, lr_base=1e-4)
    if epoch != 0 and epoch % 2 == 0:
        print("test sequence and plot trajectory!")
        # run_test(model,seq=9,dir_model = dir_model,epoch=epoch)
    model.train()
    for step, sample_t in enumerate(loader_t):
        step_global = epoch * step_per_epoch + step
        img1 = to_var(sample_t['img1'])
        img2 = to_var(sample_t['img2'])
        label_pre = model(img1, img2)

        label = to_var(sample_t['label'])
        label = label.view(-1, 6)
        loss1 = loss_func(label_pre[:, :3], label[:, :3])
        loss2 = loss_func(label_pre[:, 3:], label[:, 3:])
        loss = loss1 + args_beta * loss2

        print("epoch: {:03d} \t step: {:d} \t 'step_per_epoch:' {:d} \t t_loss: {:.4f} \t r_loss: {:.4f} loss:{:.4f}"
              .format(epoch+1, step+1, step_per_epoch, loss1, loss2, loss))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % args_epoch_save == 0:
            print(
                '\nSaving model: {:s}/model-{:d}.pkl'.format(dir_model, epoch + 1))
            torch.save(model.state_dict(), (dir_model +
                       '/model-{:d}.pkl'.format(epoch + 1)))


RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 5.81 GiB total capacity; 4.87 GiB already allocated; 10.62 MiB free; 4.89 GiB reserved in total by PyTorch)